# Crash Course to ROS 1 Python

First create the [shebang line](https://en.wikipedia.org/wiki/Shebang_(Unix)) 
and flag the file as executable

Shebang line:
```python
#!/usr/bin/env python3
```

and flag it as executable:
```bash
chmod +x {file_name}.py
```

In [1]:
#!/usr/bin/env python3

Now, import the packages we need. We of then need `numpy`, `scipy`, `rospy`, 
ros messages, and etc.

In [2]:
import rospy

import numpy as np

from std_msgs.msg import String

Note that if you import the ros messages as

```python
from std_msgs.msg import String
```

you can use the `String` object without needing to type module name, as follows:

```python
from std_msgs.msg import String

string_data = String()
```

or you can import the module and specify the object you want to use within the
module one by one, like so:

```python
import std_msgs.msg

string_data = std_msgs.msg.String()
```

Now that we include all the modules we needed, we can start writing the code.

ROS programs are called ROS nodes. Every programming language in ROS ecosystem
have their own API calls to do it. In python, we will use `rospy.init_node(...)`

> Do not forget that you need to start the ROS core in another terminal with
`roscore` command.

In [3]:
rospy.init_node("example_dumdum")

ROS is essentially a middleware framework that allows multiple programs to 
communicate with each other. It also contains open-source packages that makes
it easy to kick start and scale up the robotics projects.

To communicate with multiple programs, ROS uses **messages** that streamed with
**topics**. ROS **nodes** either **publishes** or **subscribes** to those
messages.

Lets create a subscriber first. This subscriber will only print whatever it
heard.

In [4]:
# Implement the function itself
def a_callback_function(msg):
    print("I heard this (!): {0}".format(msg.data))

# Then register the callback 
the_subscriber = rospy.Subscriber("/string_data", String, a_callback_function)

So, lets create a publisher to trigger the function `a_callback_function`.

In [5]:
the_publisher = rospy.Publisher("/string_data", String, queue_size=1)

Let's just look at what topics are published by executing `rostopic list`.

In [6]:
! rostopic list

/rosout
/rosout_agg
/string_data


Now we can call this by using `the_publisher` object.

In [7]:
string_object = String()

string_object.data = "This is the data, node the (.) to get the 'data' object."

the_publisher.publish(string_object)

I heard this (!): This is the data, node the (.) to get the 'data' object.


Note the dot notation that allowed us to access the member `data` of the 
`String` message. To understand what it really is, let's call `rosmsg show ...`
command

In [8]:
! rosmsg show std_msgs/String

string data



Well, that looks easy. But, there are indeed more complex data types. One of the
most used messages, namely, `nav_msgs/Odometry` message, is very complex and
to access the members, use should use `.`s to access objects. 

In [9]:
! rosmsg show nav_msgs/Odometry

std_msgs/Header header
  uint32 seq
  time stamp
  string frame_id
string child_frame_id
geometry_msgs/PoseWithCovariance pose
  geometry_msgs/Pose pose
    geometry_msgs/Point position
      float64 x
      float64 y
      float64 z
    geometry_msgs/Quaternion orientation
      float64 x
      float64 y
      float64 z
      float64 w
  float64[36] covariance
geometry_msgs/TwistWithCovariance twist
  geometry_msgs/Twist twist
    geometry_msgs/Vector3 linear
      float64 x
      float64 y
      float64 z
    geometry_msgs/Vector3 angular
      float64 x
      float64 y
      float64 z
  float64[36] covariance



So, then, how do you access the position of the vehicle and form your own $\eta$
vector?

$$\eta = [x, y, \psi]^\top$$

So, you do it as follows:

In [14]:
import numpy as np
from nav_msgs.msg import Odometry
from scipy.spatial.transform import Rotation as R

odom_message = Odometry()

eta = np.zeros(3)

# There is the X, Y
eta[0] = odom_message.pose.pose.position.x
eta[1] = odom_message.pose.pose.position.y

# But, ROS odometry message, stores orientation as quaternions
# So, we have to have fun to turn it into euler angles.
# 
# Right now, per initialization, all the data points are `0`s. This is zero norm
# quaternion. Therefore, it can not be transformed into euler. Let's hack it a
# bit for the example's sake.
odom_message.pose.pose.orientation.w = 1

rotation = R.from_quat([
    odom_message.pose.pose.orientation.x,
    odom_message.pose.pose.orientation.y,
    odom_message.pose.pose.orientation.z,
    odom_message.pose.pose.orientation.w])

eta[2] = rotation.as_euler("zyx")[0]